In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Library

In [ ]:
!nvidia-smi
!pip install ultralytics==8.0.196
from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from IPython.display import display, Image

In [ ]:
%cd '/content/drive/MyDrive/Wheat head Segmentation Project/Yolo8/yolov8n'

# Model train

In [ ]:
model = YOLO("/content/drive/MyDrive/Wheat head Segmentation Project/Yolo8/yolov8n/runs/segment/train3/weights/last.pt")
model.train(data="/content/drive/MyDrive/Wheat head Segmentation Project/Yolo8/Segmentation.yolov8/data_yolo8.yaml",epochs=1,imgsz=640,batch=8 , plots=True)

New https://pypi.org/project/ultralytics/8.1.20 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (AMD EPYC 7B12)
engine/trainer: task=segment, mode=train, model=/content/drive/MyDrive/Wheat head Segmentation Project/Yolo8/yolov8n/runs/segment/train3/weights/last.pt, data=/content/drive/MyDrive/Wheat head Segmentation Project/Yolo8/Segmentation.yolov8/data_yolo8.yaml, epochs=1, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, 

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c115a3b3340>
fitness: 1.4754944852845844
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'metrics/precision(M)', 'metrics/recall(M)', 'metrics/mAP50(M)', 'metrics/mAP50-95(M)']
maps: array([       1.43])
names: {0: 'wheat'}
plot: True
results_dict: {'metrics/precision(B)': 0.9133401176680178, 'metrics/recall(B)': 0.8852515985543509, 'metrics/mAP50(B)': 0.9459768799412506, 'metrics/mAP50-95(B)': 0.7610222890872328, 'metrics/precision(M)': 0.9135163620820566, 'metrics/recall(M)': 0.8828095785150515, 'metrics/mAP50(M)': 0.9389433233717768, 'metrics/mAP50-95(M)': 0.6689804497497468, 'fitness': 1.4754944852845844}
save_dir: PosixPath('runs/segment/train4')
seg: ultralytics.utils.metrics.Metric object
speed: {'preprocess': 1.463843459513

# Test Images Prediction

In [ ]:
#Predictions
!yolo task=segment mode=predict model=runs/segment/train/weights/best.pt source="/content/drive/MyDrive/Wheat head Segmentation Project/test-project#3"

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (AMD EPYC 7B12)
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs

image 1/10 /content/drive/MyDrive/Wheat head Segmentation Project/test-project#3/2fd875eaa.jpg: 640x640 33 wheats, 262.4ms
image 2/10 /content/drive/MyDrive/Wheat head Segmentation Project/test-project#3/348a992bb.jpg: 640x640 35 wheats, 195.0ms
image 3/10 /content/drive/MyDrive/Wheat head Segmentation Project/test-project#3/51b3e36ab.jpg: 640x640 26 wheats, 207.1ms
image 4/10 /content/drive/MyDrive/Wheat head Segmentation Project/test-project#3/51f1be19e.jpg: 640x640 18 wheats, 209.3ms
image 5/10 /content/drive/MyDrive/Wheat head Segmentation Project/test-project#3/53f253011.jpg: 640x640 28 wheats, 187.2ms
image 6/10 /content/drive/MyDrive/Wheat head Segmentation Project/test-project#3/796707dd7.jpg: 640x640 18 wheats, 194.4ms
image 7/10 /content/drive/MyDrive/Wheat head Segmentation Project/test-project#3/aac893a91.jpg:

#Convert best model from pt to onnx

In [ ]:
model = YOLO("/content/drive/MyDrive/Wheat head Segmentation Project/Yolo8/YOLOv8n-seg/runs/segment/train3/weights/best.pt")
results = model.predict(source="/content/drive/MyDrive/Wheat head Segmentation Project/test-project#3/2fd875eaa.jpg")[0]

model.export(format="onnx",imgsz=[640,640], opset=12)

#Inferece using onnx model

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

model_path = '/content/drive/MyDrive/Result/Yolo/Yolov8n/best.onnx'
image_path = '/content/drive/MyDrive/Wheat head Segmentation Project/test-project#3/2fd875eaa.jpg'

img = cv2.imread(image_path)
H, W, _ = img.shape

model = YOLO(model_path, task='segment')

results = model(img)
combined_mask = np.zeros((H, W), dtype=np.uint8)
polygons = []
bounding_boxes = []

for result in results:
    for j, mask in enumerate(result.masks.data):
        mask = mask.cpu().numpy() * 255
        mask = cv2.resize(mask, (W, H))

        _, binary_mask = cv2.threshold(mask, 128, 255, cv2.THRESH_BINARY)
        binary_mask = binary_mask.astype(np.uint8)
        combined_mask = np.maximum(combined_mask, binary_mask)
        contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            if cv2.contourArea(contour) > 0.5:
                epsilon = 0.02 * cv2.arcLength(contour, True)
                polygon = cv2.approxPolyDP(contour, epsilon, True)
                polygons.append(polygon)
                x, y, w, h = cv2.boundingRect(contour)
                bounding_boxes.append((x, y, x + w, y + h))

result_image = img.copy()
red_color =   (0, 255, 0)
label_color = (0, 0, 255)
thickness = 2

for box in bounding_boxes:
    x1, y1, x2, y2 = box
    cv2.rectangle(result_image, (x1, y1), (x2, y2), red_color, thickness)
    cv2.putText(result_image, 'Wheat', (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, label_color, thickness)

# Create a blank mask image
mask_image = np.zeros((H, W), dtype=np.uint8)

for polygon in polygons:
    cv2.drawContours(mask_image, [polygon], -1, 255, thickness=cv2.FILLED)

# Combine the original image with the bounding boxes and the mask
result_image = cv2.addWeighted(result_image, 0.7, cv2.cvtColor(mask_image, cv2.COLOR_GRAY2BGR), 0.3, 0)

cv2.imwrite('./Result.png', result_image)